In [1]:
#Yu Yamaoka

#Path Define
img_path = "dataset/rawdata/test.png"

#Parameter Define
model_type = 'cyto'
chan = [0,0]

In [4]:
#Function Define
from cellpose import models, io
import matplotlib.pyplot as plt

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
def img_to_cellpose(img_path):
    """
    Input:
        img_path : (string) Image file PATH
    Return:
        saved inference data at file PATH.
    """
    model = models.Cellpose(gpu=False, model_type=model_type)
    img = io.imread(img_path)
    masks, flows, styles, diams = model.eval(img, diameter=None, channels=chan)

    # save results so you can load in gui
    io.masks_flows_to_seg(img, masks, flows, diams, img_path, chan)

    # save results as png
    plt.imsave("test.png",masks)

In [3]:
#main
img_to_cellpose(img_path)